<a href="https://colab.research.google.com/github/techwack/Linear-Regression-Health-Costs-Calculator/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Importing LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Dataset and Label Encoding
df = dataset.copy()

# Creating an instance of LabelEncoder
le = LabelEncoder()

# Using .fit_transform function to fit the label encoder and return the encoded label
sex_label = le.fit_transform(df['sex'])
smoker_label = le.fit_transform(df['smoker'])
region_label = le.fit_transform(df['region'])

# Optionally, you can add these encoded labels back to the DataFrame
df['sex_encoded'] = sex_label
df['smoker_encoded'] = smoker_label
df['region_encoded'] = region_label

# Check the encoded labels
print(df[['sex', 'sex_encoded', 'smoker', 'smoker_encoded', 'region', 'region_encoded']].head())


In [ ]:
# removing the column 'sex', 'smoker', and 'region' from df
# as it is of no use now.
df.drop('sex', axis=1, inplace=True)
df.drop('smoker', axis=1, inplace=True)
df.drop('region', axis=1, inplace=True)

# appending the array to our dataframe
# with column name 'sex', 'smoker', and 'region'
df['sex'] = sex_label
df['smoker'] = smoker_label
df['region'] = region_label

# printing DataFrame
df

In [ ]:
# splitting data in training and tresting datasets
train_features = df.sample(frac=0.8, random_state=0)
test_features = df.drop(train_features.index)
train_dataset = train_features.copy()
test_dataset = test_features.copy()

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# data normalizer byt turning it into a tensor
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=['mae', 'mse'])
  return model

In [ ]:
# build and compile model
model = build_and_compile_model(normalizer)
model.summary()

In [ ]:
# Train model
history = model.fit(
    train_dataset,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=600
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
